In [ ]:
#IMPLEMENTED CODE FOR 5 SIGNS

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np   # linear algebra
import matplotlib.pylab as plt

import pandas as pd   # data processing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from keras.layers import BatchNormalization, Dropout
from keras.layers import Dense, Activation
from tensorflow import keras


from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB


plt.rcParams["figure.figsize"] = (16,10)   #Make the plots bigger by default
plt.rcParams["lines.linewidth"] = 2        #Setting the default line width
plt.style.use("ggplot")

In [ ]:
df0 = pd.read_csv("../input/emg-4/0.csv", header=None )
df1 = pd.read_csv("../input/emg-4/1.csv", header=None )
df2 = pd.read_csv("../input/emg-4/2.csv", header=None )
df3 = pd.read_csv("../input/emg-4/3.csv", header=None )
df4 = pd.read_csv("../input/ninapro.csv", header=None )   #added

df = pd.concat([df0,df1,df2,df3,df4], axis = 0)   #modified adding df4
df.head()
print(df.shape)   #used for debugging. Results: (#rows0 + … + #rows4, 65)

# rock = 0,
# scissors = 1,
# paper = 2,
# ok =3,
# sign 5 = 4

In [ ]:
x = df.loc[:,0:63]
        
y = df[64]

In [ ]:
y.unique()   # y=[0,1,2,3,4] it saves only values which are different one from each other

In [ ]:
x = np.array(x)   
y= np.array(y)

In [ ]:
x = x.reshape(x.shape[0]*x.shape[1], 1)
x.shape

In [ ]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [ ]:
x = x.reshape((-1, 8, 8))

In [ ]:
print("All Data size X and y")
print(x.shape)
print(y.shape)

In [ ]:
y = np.eye(np.max(y) + 1)[y]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42,stratify=y)

In [ ]:
print("All Data size X and y")
print(x_train.shape)
print(y_train.shape)

In [ ]:
print("Test Data size X and y")
print(x_test.shape)
print(y_test.shape)

In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 8)))   

model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50))
model.add(Dropout(0.2))

model.add(Dense(units = 64))
model.add(Dense(units = 128))

model.add(Dense(units = 5, activation="softmax"))
model.compile(optimizer = "adam" , loss = "binary_crossentropy", metrics=["accuracy"])


model.summary()

tf.keras.utils.plot_model(model, show_shapes = True, to_file="model.png")

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [ ]:
history = model.fit(x_train, y_train, epochs = 250, batch_size = 32, verbose=2 , callbacks=[callback], validation_split=0.2,)

In [ ]:
plt.figure(0)

plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [ ]:
# confusion matrix
import seaborn as sns

# Predict the values from the validation dataset:
Y_pred = model.predict(x_test)

# Convert predictions classes to one hot vectors: 
Y_pred_classes = np.argmax(Y_pred,axis = 1)   
#Returns the indices of the maximum values along axis =1.

# Convert validation observations to one hot vectors:
Y_true = np.argmax(y_test,axis = 1) 

In [ ]:
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

In [ ]:
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_true, Y_pred_classes))